## Recipe Builder Actions Overview

### Saving a File Cell
If you wish to save the contents of a cell, simply run it.  The `%%writefile` command at the top of the cell will write the contents of the cell to the file named at the top of the cell. You should run the cells manually when applicable. However, **pressing any of the actions at the top will automatically run all file cells relevant to the action**.

### Training and Scoring
Press the associated buttons at the top in order to run training or scoring. The training output will be shown below the `evaluator.py` cell and scoring output will be shown below the `datasaver.py` cell. You must run training at least once before you can run scoring. You may delete the output cell(s). Running training the first time or after changing `requirements.txt` will be slower since the dependencies for the recipe need to be installed, but subsequent runs will be significantly faster. If you wish to see the hidden output add `debug` to the end of the output cell and re-run it.

### Creating the Recipe
When you are done editing the recipe and satisfied with the training/scoring output, you can create a recipe from the notebook by pressing `Create Recipe`. You must run scoring at least once before you can create the recipe. After pressing it, you will see a progress bar showing how much time is left for the build to finish. If the recipe creation is successful the progress bar will be replaced by an external link that you can click to navigate to the created recipe.


## Caution!
* **Do not delete any of the file cells**
* **Do not edit the `%%writefile` line at the top of the file cells**

---

#### **Requirements File** (Optional)
Add additional libraries you wish to use in the recipe to the cell below. You can specify the version number if necessary. The file cell below is a **commented out example**.

In [ ]:
# pandas=0.22.0
# numpy

Search here for additional libraries https://anaconda.org/. This is the list of main **libraries already in use**:
`python=3.5.2` `scikit-learn` `pandas` `numpy` `data_access_sdk_python`
**Warning: libraries or specific versions you add may be incompatible with the above libraries**.

---

#### **Configuration Files**
List any hyperparameters you wish to use. Specify the dataset(s) and schema(s) that are needed for training/scoring. To find the dataset ids go to the **Data tab** in Adobe Experience Platform or view the **Datasets** folder in the **Notebooks Data tab** on the left. You can also find schema id in the **Notebooks Data tab** under the **Schemas** folder. Each configuration will only be used for its corresponding action. `ACP_DSW_TRAINING_XDM_SCHEMA` and `ACP_DSW_SCORING_RESULTS_XDM_SCHEMA` will only be used after the recipe has been created.

##### Training Configuration

In [ ]:
{
   "tenant_id": "<replace with tenant id>",
   "trainingDataSetId": "<replace with training dataset id>",
   "ACP_DSW_TRAINING_XDM_SCHEMA": "<replace with training xdm schema id>",
   "num_recommendations": "5"
}

##### Scoring Configuration

In [ ]:
{
   "tenant_id": "<replace with tenant id>",
   "scoringDataSetId": "<replace with scoring input dataset id>",
   "scoringResultsDataSetId": "<replace with scoring results dataset id>",
   "ACP_DSW_SCORING_RESULTS_XDM_SCHEMA": "<replace with scoring results xdm schema id>"
}

**The following configuration parameters are automatically set for you when you train/score:** 
`ML_FRAMEWORK_IMS_USER_CLIENT_ID` `ML_FRAMEWORK_IMS_TOKEN` `ML_FRAMEWORK_IMS_ML_TOKEN` `ML_FRAMEWORK_IMS_TENANT_ID`

---

#### **Training Data Loader File**
Implement the `load` function to load and prepare the training data.

In [ ]:
import numpy as np
import pandas as pd
from data_access_sdk_python.reader import DataSetReader

def load(configProperties):
    print("Training Data Load Start")
    prodreader = DataSetReader(client_id=configProperties['ML_FRAMEWORK_IMS_USER_CLIENT_ID'],
                               user_token=configProperties['ML_FRAMEWORK_IMS_TOKEN'],
                               service_token=configProperties['ML_FRAMEWORK_IMS_ML_TOKEN'])

    train_data = prodreader.load(data_set_id=configProperties['trainingDataSetId'],
                         ims_org=configProperties['ML_FRAMEWORK_IMS_TENANT_ID'])

    print("Training Data Load Finish")
    return train_data


---

#### **Scoring Data Loader File**
Implement the `load` function to load and prepare the scoring data.

In [ ]:
import numpy as np
import pandas as pd
from data_access_sdk_python.reader import DataSetReader

def load(configProperties):

    print("Scoring Data Load Start")

    prodreader = DataSetReader(client_id=configProperties['ML_FRAMEWORK_IMS_USER_CLIENT_ID'],
                               user_token=configProperties['ML_FRAMEWORK_IMS_TOKEN'],
                               service_token=configProperties['ML_FRAMEWORK_IMS_ML_TOKEN'])

    df = prodreader.load(data_set_id=configProperties['scoringDataSetId'],
                         ims_org=configProperties['ML_FRAMEWORK_IMS_TENANT_ID'])

    print("Scoring Data Load Finish")

    return df


---

#### **Pipeline File**
Implement the `train` function and return the trained model. Implement the `score` function to return a prediction made on the scoring data.

In [ ]:
import pandas as pd
import numpy as np
import itertools
import operator
from collections import Counter

class CooccurrenceBasedRecommendationModel():
    def __init__(self, configProperties):
        self.num_to_recommend = int(configProperties['num_recommendations'])
        self.recommendations = ['dummy']
        tenant_id = configProperties['tenant_id']
        self.user_id_column = '_%s.userId' % tenant_id
        self.recommendations_column = '_%s.recommendations' % tenant_id
        self.item_id_column = '_%s.itemId' % tenant_id
        self.interactions_column_name = 'interactions'
        self.cooc_matrix = {}

    def fit(self, df):
        df = df[df[self.item_id_column].notnull()]
        self.recommendations = [item for item, freq in
                                Counter(list(df[self.item_id_column].values)).most_common(self.num_to_recommend)]
        # I want to first group items by user id so that we can calculate co-occurrence
        print('grouping data by user id')
        data_grouped_by_user = df.groupby(self.user_id_column).agg(
            {self.item_id_column: lambda x: '#'.join(x)})\
            .rename(columns={self.item_id_column:self.interactions_column_name}).reset_index()

        # now calculate co-occurrence
        print('building the cooccurrence matrix')
        for index, row in data_grouped_by_user.iterrows():
            list_interactions = row[self.interactions_column_name].split('#')
            # get all pairwise combinations
            for (item_1, item_2) in list(itertools.combinations(list_interactions, 2)):
                self.update_coocur_matrix(item_1, item_2)
                self.update_coocur_matrix(item_2, item_1)

        print('fit complete')

    def update_coocur_matrix(self, item_1, item_2):
        if item_1 in self.cooc_matrix:
            # entry already exists for this item
            if item_2 in self.cooc_matrix[item_1]:
                # entry exists for this item pair
                self.cooc_matrix[item_1][item_2] += 1
            else:
                # create a new entry for this item pair
                self.cooc_matrix[item_1][item_2] = 1
        else:
            # create a new entry for this item
            self.cooc_matrix[item_1] = {}
            self.cooc_matrix[item_1][item_2] = 1


    def predict(self, df):
        # remove columns having none
        df = df[df[self.item_id_column].notnull()]

        df_grouped_by_user = df.groupby(self.user_id_column).agg(
            {self.item_id_column: lambda x: '#'.join(x)})\
        .rename(columns={self.item_id_column:self.interactions_column_name}).reset_index()

        # now for each user, we want to get recommendations based on the last seen item
        def recommend_items(row):
            list_interactions = row[self.interactions_column_name].split('#')
            last_interaction = list_interactions[-1]
            # lookup last interaction in the cooccurrence matrix
            if last_interaction in self.cooc_matrix:
                # sort cooccurrence matrix in descending order
                items_cooccurred = self.cooc_matrix[last_interaction]
                recommendations = []
                for item, cooc_count in sorted(items_cooccurred.items(), key=operator.itemgetter(0), reverse=True):
                    recommendations.append(item)
                    if len(recommendations) == self.num_to_recommend:
                        break
                # return personalized recommendations
                return '#'.join(recommendations)
            else:
                # give default recommendations
                return '#'.join(self.recommendations)


        df_grouped_by_user[self.recommendations_column] = df_grouped_by_user.apply(lambda row: recommend_items(row), axis=1)
        df_grouped_by_user = df_grouped_by_user.drop([self.interactions_column_name],axis=1)

        return df_grouped_by_user

def train(configProperties, data):

    print("Train Start")

    #########################################
    # Fit model
    #########################################
    model = CooccurrenceBasedRecommendationModel(configProperties)

    model.fit(data)

    print("Train Complete")

    return model

def score(configProperties, data, model):

    print("Score Start")

    result = model.predict(data)

    print("Score Complete")

    return result


---

#### **Evaluator File**
Implement the `split` function to partition the training data and the `evaluate` function to the return the validation metrics you wish to see. Training output will be shown below this file cell.

In [ ]:
from ml.runtime.python.Interfaces.AbstractEvaluator import AbstractEvaluator
from data_access_sdk_python.reader import DataSetReader
import numpy as np
import pandas as pd

class Evaluator(AbstractEvaluator):

    def split(self, configProperties={}, dataframe=None):
        #########################################
        # Load Data
        #########################################
        train = dataframe[:]
        test = dataframe[:]

        return train, test

    def evaluate(self, data=[], model={}, configProperties={}):
        print ("Evaluation evaluate triggered")

        # remove columns having none
        data = data[data[model.item_id_column].notnull()]

        data_grouped_by_user = data.groupby(model.user_id_column).agg(
            {model.item_id_column: lambda x: '#'.join(x)})\
        .rename(columns={model.item_id_column: model.interactions_column_name}).reset_index()

        data_recommendations = model.predict(data)

        merged_df = pd.merge(data_grouped_by_user, data_recommendations, on=[model.user_id_column]).reset_index()

        def compute_recall(row):
            set_interactions = set(row[model.interactions_column_name].split('#'))
            set_recommendations = set(row[model.recommendations_column].split('#'))
            inters = set_interactions.intersection(set_recommendations)
            if len(inters) > 0:
                return 1
            return 0

        def compute_precision(row):
            set_interactions = set(row[model.interactions_column_name].split('#'))
            list_recommendations = row[model.recommendations_column].split('#')
            score = 0
            weight = 0.5
            for rec in list_recommendations:
                if rec in set_interactions:
                    score = score + weight
                weight = weight / 2

            return score


        merged_df['recall'] = merged_df.apply(lambda row: compute_recall(row), axis=1)
        merged_df['precision'] = merged_df.apply(lambda row: compute_precision(row), axis=1)

        recall = merged_df['recall'].mean()
        precision = merged_df['precision'].mean()

        metric = [{"name": "Recall", "value": recall, "valueType": "double"},
                 {"name": "Precision", "value": precision, "valueType": "double"}]

        print(metric)

        return metric
    

---

#### **Data Saver File**
Implement the `save` function for saving your prediction. Scoring output will be added below this cell.

In [ ]:
from data_access_sdk_python.writer import DataSetWriter

def save(configProperties, prediction):
    print(prediction)
    prodwriter = DataSetWriter(client_id=configProperties['ML_FRAMEWORK_IMS_USER_CLIENT_ID'],
                               user_token=configProperties['ML_FRAMEWORK_IMS_TOKEN'],
                               service_token=configProperties['ML_FRAMEWORK_IMS_ML_TOKEN'])

    batch_id = prodwriter.write(data_set_id=configProperties['scoringResultsDataSetId'],
                 dataframe=prediction,
                 ims_org=configProperties['ML_FRAMEWORK_IMS_TENANT_ID'])
    print("Data written successfully to platform:",batch_id)
